# 1. Import and Install Dependencies

## 1.1 Install Dependencies

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install tensorflow tensorflow-gpu  matplotlib
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1.2 Load Dependencies

In [ ]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio

# 2. Build Data Loading Function

## 2.2 Build Dataloading Function

In [ ]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

# 3. Create Tensorflow Dataset

## 3.1 Define Paths to Positive and Negative Data

In [ ]:
def return_slices(FILEPATH):
    wav = load_mp3_16k_mono(FILEPATH)
    audio_slices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=11200, sequence_stride=11200, batch_size=1)
    return audio_slices

In [ ]:
#森田

person = 13 # 

SS = [[] for i in range(person)]

for i in range(person + 1):
  if i < 10:
    # 
    SS[i-1] = os.path.join('/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw', 's0' + str(i))
  else:
    # 
    SS[i-1] = os.path.join('/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw', 's' + str(i))


## 3.2 Create Tensorflow Datasets

In [ ]:
#森田2
import os

ss = [[] for i in range(person)]

for n in range(person):

  for i in os.listdir(SS[n]):
    # CALCULATE NUM
    #print(SS[n] + '/' + i)
    wav = load_wav_16k_mono(SS[n] + '/' + i)
    wav = wav[:int(len(wav) * 0.7)]

    # !!!!!!!!!!!!!!!!!!!!!!!!!!短縮版！！！！！！！！！！！！！！
    NUM = int(len(wav) / (0.75 * 16000))
    #print(NUM)

    for j in range(NUM):
      ss[n].append(os.path.join(SS[n],i+str(j)))
      
  #print(ss[n])
  ss[n] = tf.data.Dataset.from_tensor_slices(ss[n])


## 3.3 Add labels and Combine Positive and Negative Samples

In [ ]:
#森田
# data = tf.data.Dataset()
sis = []
for i in range(13):

  indices = i
  depth = 13  

  sis.append( tf.data.Dataset.zip((ss[i], tf.data.Dataset.from_tensor_slices(([tf.one_hot(indices, depth)]*len(ss[i]))))) )

for i in range(len(sis)-1):
  sis[0] = sis[0].concatenate(sis[i+1])
data = sis[0]

print(len(data))

print(data.as_numpy_iterator().next())

10646
(b'/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw/s01/s01_2_footstep_audio.wav0', array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))


# 4. Build Preprocessing Function to Convert to Spectrogram

## 5.1 Build Preprocessing Function

In [ ]:

def preprocess(file_path, label):
    num = ""
    for i in reversed(file_path):
      if 48<=i<=57:
        num = chr(i) + num
      else:
        break
    wav = load_wav_16k_mono(file_path[:-len(num)])
    num = int(num)
    
    ###
    wav_clip = wav[12000*num:12000+12000*num]
    spectrogram = tf.signal.stft(wav_clip, frame_length=320, frame_step=160)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label


# 5. Create Training and Testing Partitions

## 5.1 Create a Tensorflow Data Pipeline

In [ ]:
# positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.one_hot(indices=[0]*len(pos),depth=2,on_value=1.0,off_value=0,axis=-1,dtype=tf.float32))))
# negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.one_hot(indices=[1]*len(neg),depth=2,on_value=1.0,off_value=0,axis=-1,dtype=tf.float32))))
# # positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
# # negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
# data = positives.concatenate(negatives)

import numpy as np
data_numpy = []
for i in data:
  data_numpy.append([i[0].numpy(), i[1].numpy()])

data_numpy_processed_X = []
data_numpy_processed_y = []

for i in data_numpy:
  spectrogram, label = preprocess(i[0],i[1])
  data_numpy_processed_X.append(spectrogram.numpy())
  data_numpy_processed_y.append(label)

  # spectrogram
print(len(data_numpy_processed_X))

# data = data.map(preprocess)
# data_numpy = [[]]
# # for i in data.take(1):
# #   # print(i[0].numpy())
# #   print(i[1].numpy())

# # 問題のところ、データセットの次元を変えたい。（flattenしたい）
# # data = data.flat_map(tf.data.Dataset.from_tensor_slices)
# data = data.cache()
# data = data.shuffle(buffer_size=1000)
# data = data.batch(16)
# data = data.prefetch(8)
# print(len(data))


## 5.2 Split into Training and Testing Partitions

In [ ]:
# train = data.take(36)
# test = data.skip(36).take(15)
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_numpy_processed_X, data_numpy_processed_y, test_size=0.2)
train_X = np.array(train_X)
train_y = np.array(train_y)
test_X = np.array(test_X)
test_y = np.array(test_y)

# # 正規化
# scaler = StandardScaler()
# scaler.fit(data)
# newdata = scaler.transform(data)

# data_numpy_processed_X = np.array(data_numpy_processed_X)
# print(data_numpy_processed_X.shape)

# train_X = np.array(data_numpy_processed_X)
# train_y = np.array(data_numpy_processed_y)

# # shuffle
# print(train_X.shape)
# indices = np.arange(train_X.shape[0])
# np.random.shuffle(indices)
# train_X = train_X[indices]
# train_y = train_y[indices]

print(train_X.shape)
print(train_y.shape)

print(np.max(train_X))
print(np.min(train_X))
print()
train_X = train_X / np.max(train_X)
# save the data
print(np.max(train_X))
np.save("train_X",train_X)
np.save("train_y",train_y)

##5.3　もし以前の変数が保存されたら、データを読み込む

In [ ]:
# もしデータがすでに保存されているなら、直接読み込みます
import os
import numpy as np
type_folder = "step_160_data_1"
if os.path.exists("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/train_X.npy"):
  train_X = np.load("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/train_X.npy")
  train_y = np.load("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/train_y.npy")
  print("Yes")

# 6. Build Deep Learning Model

## 6.1 Load Tensorflow Dependencies

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout

## 6.2 Build Sequential Model, Compile and View Summary

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(train_X.shape[1], 257,1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(32, activation='tanh'))
model.add(Dense(13, activation='softmax'))

In [ ]:
model.compile('Adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.summary()

## 6.3 Fit Model, View Loss and KPI Plots

In [ ]:
hist = model.fit(train_X, train_y, epochs=14, validation_split=0.2, batch_size = 32)   
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model")

In [ ]:
print(type(hist))
print(hist.history.keys())

In [ ]:
plt.title('Loss')
plt.plot(hist.history['loss'], 'r',label="train")
plt.plot(hist.history['val_loss'], 'b', label="test")
plt.legend()
plt.show()

In [ ]:
plt.title('accuracy')
plt.plot(hist.history['accuracy'], 'r', label="train")
plt.plot(hist.history['val_accuracy'], 'b', label="validation")
plt.legend()

plt.show()


# 7.テストデータに対し精度を計算、


## モデルを直接読み込む

In [ ]:
from tensorflow import keras
import os
# It can be used to reconstruct the model identically.
type_folder = "step_160_data_1"
if os.path.exists("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/my_model"):
  model = keras.models.load_model("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/my_model")
  print("Yes")

Yes


## テストデータを作る/読み込む

In [ ]:
#森田
import os

t01 = []
t02 = []
t03 = []
t04 = []
t05 = []
t06 = []
t07 = []
t08 = []
t09 = []
t10 = []
t11 = []
t12 = []
t13 = []

ts = [t01, t02, t03, t04, t05, t06, t07, t08, t09, t10, t11, t12, t13]

for n in range(13):
    
  for i in os.listdir(SS[n]):
    # CALCULATE NUM
    wav = load_wav_16k_mono(SS[n] + '/' + i)
    # wav = wav[int(len(wav) * 0.7):]
    
    NUM = int(len(wav)*0.3 / (10 * 16000))

    for j in range(NUM):
      ts[n].append(os.path.join(SS[n],i+str(j)))


  ts[n] = tf.data.Dataset.from_tensor_slices(ts[n])


In [ ]:
#森田
# data = tf.data.Dataset()
tis = []
for i in range(13):

  indices = i
  depth = 13    

  tis.append( tf.data.Dataset.zip((ts[i], tf.data.Dataset.from_tensor_slices(([tf.one_hot(indices, depth)]*len(ts[i]))))) )

for i in range(len(tis)-1):
  tis[0] = tis[0].concatenate(tis[i+1])
data_test = tis[0]
#positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
#negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
#data = positives.concatenate(negatives)

print(len(data_test))

print(data_test.as_numpy_iterator().next())

In [ ]:
def preprocess(file_path, label):
    num = ""
    for i in reversed(file_path):
      if 48<=i<=57:
        num = chr(i) + num
      else:
        break
    wav = load_wav_16k_mono(file_path[:-len(num)])

    num = int(num)
    print(num)

    wav_clip = wav[int(len(wav) * 0.7)+160_000*num:int(len(wav) * 0.7)+ 160_000+160_000*num]
    length_clip_clip = int(0.75 * 16_000)
    spectrogram_list = []
    # print(int(len(wav) * 0.7)+ 160_000*num, int(len(wav) * 0.7)+ 160_000+160_000*num)
    for i in range(int(len(wav_clip)/length_clip_clip)):
      wav_clip_clip = wav_clip[length_clip_clip * i: length_clip_clip + length_clip_clip * i]
      spectrogram = tf.signal.stft(wav_clip_clip, frame_length=320, frame_step=160)
      spectrogram = tf.abs(spectrogram)
      spectrogram = tf.expand_dims(spectrogram, axis=2)
      spectrogram_list.append(spectrogram.numpy())
    return spectrogram_list, label

In [ ]:
import numpy as np
data_numpy_test = []
for i in data_test:
  data_numpy_test.append([i[0].numpy(), i[1].numpy()])

data_numpy_processed_X_test = []
data_numpy_processed_y_test = []

for i in data_numpy_test:
  spectrogram, label = preprocess(i[0],i[1])
  data_numpy_processed_X_test.append(spectrogram)
  data_numpy_processed_y_test.append(label)

  # spectrogram

test_X = np.array(data_numpy_processed_X_test)
test_y = np.array(data_numpy_processed_y_test)

# shuffle
print(test_X.shape)
indices = np.arange(test_X.shape[0])
np.random.shuffle(indices)
test_X = test_X[indices]
test_y = test_y[indices]

print(test_X.shape)
print(test_y.shape)
print(test_X.shape[1])

# save the data
np.save("test_X",test_X)
np.save("test_y",test_y)



テストデータを直接読み込む

In [ ]:
import os
import numpy as np
type_folder = "step_160_data_1"
if os.path.exists("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/test_X.npy"):
  test_X = np.load("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/test_X.npy")
  test_y = np.load("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/test_y.npy")
  print("Yes")
print(test_X.shape)
print(test_y.shape)
print(np.max(test_X))

Yes
(390, 13, 74, 257, 1)
(390, 13)
4.53607


## 精度を計算する

In [ ]:
# テストの精度を計算
sum_acc = 0
print(test_X.shape)
print(type(test_X[1][1]))
for index in range(len(test_X)):
  res_pos = model.predict(test_X[index])
  result_num = np.argmax(res_pos.sum(axis=0))
  if result_num == np.argmax(test_y[index]):
    sum_acc += 1
print("accuracy", sum_acc/len(test_y))

(390, 13, 74, 257, 1)
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 24ms/step
accuracy 0.9897435897435898


# 8.音声を読み込み、テストを行う

In [ ]:
file_path = "/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-Raw/S04/S04_6_footstep_audio.wav"
wav = load_wav_16k_mono(file_path)

# spectrogramを計算する
num = 2 # wavのどの10sの部分を切り取るか
wav_clip = wav[int(len(wav) * 0.7)+160_000*num:int(len(wav) * 0.7)+ 160_000+160_000*num]
length_clip_clip = int(0.75 * 16_000) # 0.75秒
spectrogram_list = []
# print(int(len(wav) * 0.7)+ 160_000*num, int(len(wav) * 0.7)+ 160_000+160_000*num)
for i in range(int(len(wav_clip)/length_clip_clip)):
  wav_clip_clip = wav_clip[length_clip_clip * i: length_clip_clip + length_clip_clip * i]
  spectrogram = tf.signal.stft(wav_clip_clip, frame_length=320, frame_step=160)
  spectrogram = tf.abs(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis=2)
  spectrogram_list.append(spectrogram.numpy())

spectrogram_list = np.array(spectrogram_list)
print(spectrogram_list.shape)
res_pos = model.predict(spectrogram_list)
result_num = np.argmax(res_pos.sum(axis=0))

print("classification ",np.argmax(res_pos.sum(axis=0))+1)
  